In [1]:
import ipynb.fs.full.utils as utils
import ipynb.fs.full.blockchain as blockchain

### LOAD RESOURCES

In [2]:
settings = utils.load_yaml('config/settings.yaml')

In [3]:
latest = utils.load_json('config/latest.json')

### CONNECT TO ETHEREUM GATEWAY

In [4]:
web3 = blockchain.connect(settings)

### SERIALIZE MANAGER CONTRACTS

In [5]:
user_manager = blockchain.contract(latest['usermanager'], web3, settings)

In [6]:
oracle_manager = blockchain.contract(latest['oraclemanager'], web3, settings)

In [7]:
task_manager = blockchain.contract(latest['taskmanager'], web3, settings)

In [8]:
token_manager = blockchain.contract(latest['tokenmanager'], web3, settings)

### INIT TOKEN MANAGER

In [9]:
token_symbol = 'ArcaCoin'
token_price = 5000
token_capacity = 10000

In [10]:
token_manager.write({
    'func': 'init',
    'params': [
        token_symbol,
        token_price,
        token_capacity,
        task_manager.address
    ]
})

web3.exceptions.SolidityError('execution reverted: contract has already been initialized')

### INIT TASK MANAGER

In [11]:
task_fee = 2

In [12]:
task_manager.write({
    'func': 'init',
    'params': [
        task_fee,
        user_manager.address,
        oracle_manager.address,
        token_manager.address
    ]
})

web3.exceptions.SolidityError('execution reverted: contract has already been initialized')

### INIT USER MANAGER

In [13]:
user_manager.write({
    'func': 'init',
    'params': [task_manager.address]
})

web3.exceptions.SolidityError('execution reverted: contract has already been initialized')

### INIT ORACLE MANAGER

In [14]:
oracle_manager.write({
    'func': 'init',
    'params': [
        user_manager.address,
        task_manager.address
    ]
})

web3.exceptions.SolidityError('execution reverted: contract has already been initialized')